In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.preprocessing import LabelEncoder  
import numpy_indexed as npi
import scipy.io
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [6]:
total_sample = pd.read_csv('./outputs/total_sample.csv',index_col=0)
total_sample.columns = ['user','track','rating']
print(total_sample.head())
print(total_sample.shape)

             user      track  rating
11789948  9310301  9502431.0       1
11789882  9310301  8990956.0       1
11789884  9310301  9045799.0       1
11789885  9310301  9045807.0       1
11789915  9310301  9158953.0       1
(79847, 3)


In [7]:
tracks_sample_unique = total_sample['track'].unique()

In [8]:
track_tag=pd.read_csv('./outputs/track_tag_df.csv',index_col=0)
track_tag.head()

,TRACK_ID,TAG_ID,TYPE_KEY
0,2828445,379950871,GENRE
1,2828445,379950872,GENRE
2,2828445,379950903,CURATION
3,2828445,379950950,MOOD
4,2828445,379950952,MOOD


In [5]:
track_tag_sample = track_tag.loc[track_tag['TRACK_ID'].isin(tracks_sample_unique)]
print(track_tag_sample['TRACK_ID'].nunique())

1000


In [6]:
track_tag_sample['TYPE_KEY'].unique()

array(['GENRE', 'MOOD', 'ORCHESTRATION', 'CURATION', 'FORM'], dtype=object)

In [7]:
track_tag_sample = track_tag_sample.sort_values(by=['TRACK_ID'])
track_tag_sample.head()

,TRACK_ID,TAG_ID,TYPE_KEY
29781,2832434,379950871,GENRE
29782,2832434,379950872,GENRE
29783,2832434,379950952,MOOD
29784,2832434,379950971,MOOD
29785,2832434,379951104,ORCHESTRATION


In [8]:
tracks_sample_unique = track_tag_sample['TRACK_ID'].unique()
tracks_sample_unique

array([  2832434,   2834013,   2850931,   2852012,   2867883,   2867903,
         2868044,   2868273,   2868779,   2869120,   2869197,   2869370,
         2869481,   2869574,   2869611,   2869633,   2869658,   2869663,
         2869664,   2869665,   2869666,   2869667,   2869668,   2869669,
         2869671,   2869683,   2869754,   2869892,   2869893,   6036974,
         6036986,   6037213,   6044083,   6044228,   6048990,   6049027,
         6049034,   6049095,   6049100,   6049160,   6049167,   6049181,
         6049187,   6049200,   6049227,   6049287,   6049329,   6049340,
         6049349,   6049360,   6049367,   6058520,   6058523,   6058524,
         6058525,   6058526,   6058527,   6058528,   6058529,   6058530,
         6058531,   6058532,   6058533,   6058534,   6058535,   6058592,
         6058593,   6058594,   6058595,   6058596,   6058597,   6058598,
         6058599,   6058600,   6058601,   6058602,   6058603,   6058604,
         6058605,   6058606,   6058607,   6059178, 

In [9]:
groupby_typekey=track_tag_sample.groupby(['TYPE_KEY'])

In [10]:
print('Genre unique: ',groupby_typekey.get_group('GENRE')['TAG_ID'].nunique())
print('CURATION unique: ',groupby_typekey.get_group('CURATION')['TAG_ID'].nunique())
print('MOOD unique: ',groupby_typekey.get_group('MOOD')['TAG_ID'].nunique())
print('ORCHESTRATION unique: ',groupby_typekey.get_group('ORCHESTRATION')['TAG_ID'].nunique())
print('FORM unique: ',groupby_typekey.get_group('FORM')['TAG_ID'].nunique())

Genre unique:  23
CURATION unique:  23
MOOD unique:  33
ORCHESTRATION unique:  62
FORM unique:  2


In [11]:
def feature_Build(groupby_typekey,col):
    feature = groupby_typekey.get_group(col)
    le = LabelEncoder()
    feature['TAG_ID'] = le.fit_transform(feature['TAG_ID'])
    feature.drop(['TYPE_KEY'], axis = 1, inplace = True)

    feature = pd.get_dummies(feature, columns=['TAG_ID'])
    feature_cols= feature.columns.drop(['TRACK_ID'])
    feature=feature.groupby(['TRACK_ID'])[feature_cols].sum().reset_index()
#     feature.drop(['TRACK_ID'], axis = 1, inplace = True)
    return feature

In [12]:
genre = feature_Build(groupby_typekey,'GENRE' )
print(genre.shape)
genre.tail()

(1000, 24)


/Users/macbookpro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/macbookpro/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,TRACK_ID,TAG_ID_0,TAG_ID_1,TAG_ID_2,TAG_ID_3,TAG_ID_4,TAG_ID_5,TAG_ID_6,TAG_ID_7,TAG_ID_8,...,TAG_ID_13,TAG_ID_14,TAG_ID_15,TAG_ID_16,TAG_ID_17,TAG_ID_18,TAG_ID_19,TAG_ID_20,TAG_ID_21,TAG_ID_22
995,555361530,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
996,555361558,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
997,555361620,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
998,555361656,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
999,555361728,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
curation = feature_Build(groupby_typekey,'CURATION' )
curation.shape

/Users/macbookpro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(243, 24)

In [14]:
mood = feature_Build(groupby_typekey,'MOOD' )
mood.shape

/Users/macbookpro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(970, 34)

In [15]:
orchestration = feature_Build(groupby_typekey,'ORCHESTRATION' )
orchestration.shape

/Users/macbookpro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(1000, 63)

In [16]:
form = feature_Build(groupby_typekey,'FORM' )
form.shape

/Users/macbookpro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(13, 3)

In [17]:
user_track = total_sample.pivot_table('rating','track','user',fill_value=0,dropna=False)
user_track = user_track.sort_values('track')
print(user_track.shape)
user_track.tail()

(1000, 100)


user,3568920,3568922,3568924,3568932,3568998,3581842,3582502,3582854,3582874,3593458,...,62063406,64359968,68849385,77495305,116965066,153428933,222759474,239010771,339744862,488313775
track,,,,,,,,,,,,,,,,,,,,,
555361530.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,1,0,0
555361558.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,1,0,0
555361620.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,1,0,0
555361656.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,1,0,0
555361728.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,1,0,0


In [18]:
gt_cosine_sim = cosine_similarity(user_track,user_track)
print(gt_cosine_sim.shape)
print(gt_cosine_sim[(gt_cosine_sim>=0.999) & (gt_cosine_sim<=1.001)].sum()/2)
gt_cosine_sim[:5,:5]

(1000, 1000)
724.0000000000001


array([[1.        , 0.58304494, 0.21155296, 0.40406881, 0.27993145],
       [0.58304494, 1.        , 0.29813649, 0.42361111, 0.42454476],
       [0.21155296, 0.29813649, 1.        , 0.31348769, 0.45190311],
       [0.40406881, 0.42361111, 0.31348769, 1.        , 0.37737312],
       [0.27993145, 0.42454476, 0.45190311, 0.37737312, 1.        ]])

In [19]:
gt = user_track.loc[user_track.index.isin(genre['TRACK_ID'].unique())].sort_index()
genre = genre.drop('TRACK_ID',axis=1)
gt_cosine_sim = cosine_similarity(gt,gt)
cosine_sim_genre = cosine_similarity(genre, genre)
print(cosine_sim_genre.shape)
print(cosine_sim_genre[(cosine_sim_genre>=0.999) & (cosine_sim_genre<=1.001)].sum()//2+1)
print(r2_score(gt_cosine_sim.flatten(),cosine_sim_genre.flatten()))
print(mean_squared_error(cosine_sim_genre.flatten(),gt_cosine_sim.flatten()))
cosine_sim_genre[:10,:10]

(1000, 1000)
297800.0
-6.613156436056851
0.20791837166851776


array([[1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 1., 1.],
       [1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 1., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 1., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 1., 0., 1., 1., 1., 1.]])

In [20]:
gt = user_track.loc[user_track.index.isin(mood['TRACK_ID'].unique())].sort_index()
mood = mood.drop('TRACK_ID',axis=1)
cosine_sim_mood = cosine_similarity(mood, mood)
gt_cosine_sim = cosine_similarity(gt,gt)
print(cosine_sim_mood.shape)
print(cosine_sim_mood[(cosine_sim_mood>=0.999) & (cosine_sim_mood<=1.001)].sum()//2 + 1)
print(r2_score(gt_cosine_sim.flatten(),cosine_sim_mood.flatten()))
print(mean_squared_error(gt_cosine_sim.flatten(),cosine_sim_mood.flatten()))
cosine_sim_mood[:5,:5]

(970, 970)
9136.0
-12.152456775523527
0.3473287239642571


array([[1.        , 0.        , 0.        , 0.40824829, 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.70710678],
       [0.        , 0.        , 1.        , 0.        , 0.        ],
       [0.40824829, 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.70710678, 0.        , 0.        , 1.        ]])

In [21]:
gt = user_track.loc[user_track.index.isin(orchestration['TRACK_ID'].unique())].sort_index()
orchestration = orchestration.drop('TRACK_ID',axis=1)
gt_cosine_sim = cosine_similarity(gt,gt)
cosine_sim_orchestration = cosine_similarity(orchestration, orchestration)
print(cosine_sim_orchestration.shape)
print(cosine_sim_orchestration[(cosine_sim_orchestration>=0.999) & (cosine_sim_orchestration<=1.001)].sum()//2 + 1)
print(r2_score(gt_cosine_sim.flatten(),cosine_sim_orchestration.flatten()))
print(mean_squared_error(gt_cosine_sim.flatten(),cosine_sim_orchestration.flatten()))
cosine_sim_orchestration[:5,:10]

(1000, 1000)
50567.0
-1.1107881820859946
0.05764647626021717


array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        0.57735027, 0.70710678, 0.70710678, 0.70710678, 0.70710678],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        0.57735027, 0.70710678, 0.70710678, 0.70710678, 0.70710678],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        0.57735027, 0.70710678, 0.70710678, 0.70710678, 0.70710678],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        0.57735027, 0.70710678, 0.70710678, 0.70710678, 0.70710678],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        0.57735027, 0.70710678, 0.70710678, 0.70710678, 0.70710678]])

In [22]:
gt = user_track.loc[user_track.index.isin(curation['TRACK_ID'].unique())].sort_index()
curation = curation.drop('TRACK_ID',axis=1)
gt_cosine_sim = cosine_similarity(gt,gt)
cosine_sim_curation = cosine_similarity(curation, curation)
print(cosine_sim_curation.shape)
print(cosine_sim_curation[(cosine_sim_curation>=0.999) & (cosine_sim_curation<=1.001)].sum()//2 + 1)
print(r2_score(gt_cosine_sim.flatten(),cosine_sim_curation.flatten()))
print(mean_squared_error(gt_cosine_sim.flatten(),cosine_sim_curation.flatten()))
cosine_sim_curation[:10,:10]

(243, 243)
2998.0
-15.162312033910577
0.42529747409340274


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [23]:
gt = user_track.loc[user_track.index.isin(form['TRACK_ID'].unique())].sort_index()
form = form.drop('TRACK_ID',axis=1)
gt_cosine_sim = cosine_similarity(gt,gt)
cosine_sim_form = cosine_similarity(form, form)
print(cosine_sim_form.shape)
print(cosine_sim_form[cosine_sim_form==1].sum())
print(r2_score(gt_cosine_sim.flatten(),cosine_sim_form.flatten()))
print(mean_squared_error(gt_cosine_sim.flatten(),cosine_sim_form.flatten()))
cosine_sim_form[:10,:10]

(13, 13)
125.0
-1.9163741354070911
0.374324894729173


array([[1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.]])